In [7]:
!pip install pvlib pandas numpy

In [29]:
import requests
import pandas as pd
import numpy as np
import pvlib
from pvlib import location, pvsystem, modelchain, irradiance, solarposition
import json
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

# Parâmetros de temperatura SAPM (Sandia Array Performance Model)
# SAPM é um modelo empírico que correlaciona temperatura da célula com:
# - Irradiância no plano do módulo [W/m²]  
# - Temperatura ambiente [°C]
# - Velocidade do vento [m/s]
# Equação: T_cell = T_air + (irradiance/1000) * deltaT
# onde deltaT depende da tecnologia e montagem (open_rack_glass_glass ≈ 3°C)
temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

def process_your_pvgis_structure():
    """
    Processador específico para a estrutura de dados que você recebe:
    {
        "time":"20200101:0003",    # Timestamp UTC no formato YYYYMMDD:HHMM
        "G(i)":0.0,                # Irradiância global no plano inclinado [W/m²]
        "H_sun":0.0,               # Elevação solar [graus acima do horizonte]
        "T2m":25.02,               # Temperatura do ar a 2m de altura [°C]
        "WS10m":1.93,              # Velocidade do vento a 10m [m/s] 
        "Int":0.0                  # Flag de interpolação de dados (0=medido, 1=interpolado)
    }
    
    PVLIB Framework Integration:
    - Utiliza modelos validados cientificamente para simulação PV
    - Implementa cadeia completa: irradiância → temperatura → I-V → potência
    - Considera efeitos meteorológicos, geométricos e elétricos
    """
    
    # Coordenadas do sistema fotovoltaico (Umuarama, Paraná)
    # Latitude negativa: hemisfério sul (importante para cálculos astronômicos)
    lat, lon = -23.761759, -53.329072
    
    print("🔧 PROCESSADOR PARA SUA ESTRUTURA PVGIS ESPECÍFICA")
    print("=" * 60)
    print(f"📍 Coordenadas: {lat}, {lon}")
    
    # URL exata para seus dados - USANDO ANGLE=0 para GHI - ANO 2020
    # angle=0: obtém GHI (Global Horizontal Irradiance) para transposição via pvlib
    # usehorizon=1: considera sombreamento por topografia/horizonte local
    # selectrad=1: inclui dados detalhados de radiação solar
    url = f"https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?lat={lat}&lon={lon}&startyear=2020&endyear=2020&outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0"
    
    print(f"🌐 URL: {url}")
    print("⚠️  IMPORTANTE: angle=0 para obter GHI e usar pvlib para transposição")
    
    try:
        print("\n🔄 Fazendo requisição...")
        response = requests.get(url, timeout=60)
        data = response.json()
        
        # Extrai dados hourly
        hourly_data = data['outputs']['hourly']
        print(f"✅ Recebidos {len(hourly_data):,} registros")
        
        # Mostra primeiros registros para verificação
        print(f"\n📊 PRIMEIROS 3 REGISTROS:")
        for i, record in enumerate(hourly_data[:3]):
            print(f"   Registro {i+1}: {record}")
        
        # Processamento específico para sua estrutura
        processed_data = process_your_specific_format(hourly_data, lat, lon)
        
        if processed_data is not None:
            # Análise dos dados
            analyze_your_data(processed_data)
            
            # Simulação PV USANDO PVLIB - CONFIGURAÇÃO ESPECÍFICA
            print(f"\n" + "="*50)
            print("🎯 SIMULAÇÃO COM ORIENTAÇÃO ESPECÍFICA:")
            simulate_with_your_data(
                processed_data, lat, lon,
                module_power=550,      # Potência nominal STC do módulo [Wp]
                num_modules=18,        # Número de módulos em série (string)
                surface_tilt=24,       # Inclinação do array [graus] - otimizada para latitude
                surface_azimuth=90    # Azimute [graus] - 180°=Norte (hemisfério sul)
            )
            
            # OPCIONAL: Teste múltiplas orientações
            print(f"\n" + "="*50)  
            print("🔍 TESTE DE MÚLTIPLAS ORIENTAÇÕES:")
            print("   (Para encontrar a orientação ótima)")
            
            # Descomente a linha abaixo para testar orientações:
            optimization_results = test_orientations(processed_data, lat, lon)
            
        return processed_data
        
    except Exception as e:
        print(f"❌ Erro na requisição: {e}")
        return None

def process_your_specific_format(hourly_data, lat, lon):
    """
    Processa especificamente os campos que você recebe
    CONVERTENDO PARA FORMATO PVLIB
    
    Transformação de dados PVGIS → pvlib:
    - Parse temporal: timestamps PVGIS → datetime pandas com timezone
    - Mapeamento de variáveis: nomenclatura PVGIS → padrão pvlib
    - Validação física: verificação de limites e consistência
    - Preparação para modelchain: estrutura de dados compatível
    """
    print(f"\n🔧 PROCESSANDO ESTRUTURA ESPECÍFICA PARA PVLIB:")
    print("   Campos disponíveis: time, G(i), H_sun, T2m, WS10m, Int")
    print("   Convertendo para: ghi, dni, dhi, temp_air, wind_speed")
    
    processed_records = []
    
    for record in hourly_data:
        try:
            # Parse do timestamp: "20200101:0003"
            time_str = record['time']
            
            # Extração manual mais robusta dos componentes temporais
            year = int(time_str[0:4])      # 2020
            month = int(time_str[4:6])     # 01  
            day = int(time_str[6:8])       # 01
            hour = int(time_str[9:11])     # 00
            minute = int(time_str[11:13])  # 03
            
            # Cria datetime com timezone UTC (padrão PVGIS)
            dt = pd.Timestamp(year=year, month=month, day=day, 
                            hour=hour, minute=minute, tz='UTC')
            
            # Extrai seus campos específicos e mapeia para padrão pvlib
            processed_record = {
                'datetime': dt,
                'ghi': float(record['G(i)']),             # G(i) com angle=0 ≈ GHI [W/m²]
                'sun_elevation': float(record['H_sun']),  # Elevação solar [graus]
                'temp_air': float(record['T2m']),         # Temperatura do ar [°C]
                'wind_speed': float(record['WS10m']),     # Velocidade do vento [m/s]
                'interpolated': float(record['Int']),     # Flag interpolação PVGIS
                
                # Campos extras para pvlib (valores padrão quando não disponíveis)
                'pressure': 101325.0,  # Pressão atmosférica padrão ao nível do mar [Pa]
                
                # Para análise temporal e debugging
                'year': year,
                'month': month,
                'day': day, 
                'hour': hour,
                'minute': minute
            }
            
            processed_records.append(processed_record)
            
        except (KeyError, ValueError) as e:
            print(f"⚠️  Erro processando registro: {e}")
            continue
    
    if not processed_records:
        print("❌ Nenhum registro processado com sucesso")
        return None
    
    # Cria DataFrame com índice temporal (fundamental para pvlib)
    df = pd.DataFrame(processed_records)
    df.set_index('datetime', inplace=True)
    
    # Converte para timezone local brasileiro (UTC-3 padrão, UTC-2 horário de verão)
    df.index = df.index.tz_convert('America/Sao_Paulo')
    
    print(f"✅ Processados {len(df):,} registros com sucesso")
    print(f"   Período: {df.index.min()} até {df.index.max()}")
    
    # CRUCIAL: Decompor GHI usando PVLIB
    # Necessário porque pvlib precisa das 3 componentes (GHI, DNI, DHI) 
    # para transposição precisa no plano inclinado
    df = decompose_ghi_with_pvlib(df, lat, lon)
    
    return df

def decompose_ghi_with_pvlib(df, lat, lon):
    """
    USA PVLIB para decompor GHI em componentes DNI e DHI
    
    Modelo DISC (Direct Insolation Simulation Code):
    - Baseado no índice de clareza atmosférica: kt = GHI / ETR
    - Utiliza correlações empíricas validadas para estimar DNI
    - Considera massa de ar óptica e geometria solar
    
    Componentes de irradiância solar:
    - GHI: Global Horizontal Irradiance (total no plano horizontal)
    - DNI: Direct Normal Irradiance (direta perpendicular aos raios)
    - DHI: Diffuse Horizontal Irradiance (difusa por espalhamento)
    
    Relação física fundamental: GHI = DNI × cos(θz) + DHI
    onde θz é o ângulo zenital solar
    """
    print(f"\n🔬 DECOMPOSIÇÃO GHI → DNI/DHI COM PVLIB:")
    
    try:
        # Posição solar usando algoritmo SPA (Solar Position Algorithm)
        # SPA: precisão ±0.0003° para período 2000-6000 DC
        # Considera refração atmosférica, nutação, aberração
        solar_position = solarposition.get_solarposition(
            df.index, lat, lon,
            pressure=df['pressure'],      # Correção por pressão atmosférica
            temperature=df['temp_air']    # Correção por refração atmosférica
        )
        
        print(f"   ☀️ Posição solar calculada")
        
        # Decompõe GHI usando modelo DISC da pvlib
        # DISC utiliza correlações entre clearness index e fração DNI/GHI
        # Entrada: GHI medido, ângulo zenital solar, data/hora
        # Saída: DNI estimado com base em modelos empíricos validados
        decomposed = irradiance.disc(
            ghi=df['ghi'],
            solar_zenith=solar_position['zenith'],
            datetime_or_doy=df.index
        )
        
        # O modelo DISC retorna DataFrame/Series - vamos verificar as chaves
        print(f"   🔍 Chaves disponíveis na decomposição: {decomposed.columns.tolist() if hasattr(decomposed, 'columns') else type(decomposed)}")
        
        # Adiciona componentes (usando as chaves corretas)
        if isinstance(decomposed, pd.DataFrame):
            df['dni'] = decomposed['dni']
            # Calcula DHI pela relação física: DHI = GHI - DNI × cos(θz)
            df['dhi'] = df['ghi'] - df['dni'] * np.cos(np.radians(solar_position['zenith']))
        else:
            # Se for Series, é o DNI diretamente
            df['dni'] = decomposed
            df['dhi'] = df['ghi'] - df['dni'] * np.cos(np.radians(solar_position['zenith']))
        
        # Adiciona posição solar ao dataset (necessário para transposição)
        df['solar_zenith'] = solar_position['zenith']    # Ângulo zenital [graus]
        df['solar_azimuth'] = solar_position['azimuth']  # Azimute solar [graus]
        
        # Garante valores físicamente possíveis (irradiâncias não-negativas)
        df['dni'] = df['dni'].clip(lower=0)
        df['dhi'] = df['dhi'].clip(lower=0)
        
        print(f"   ✅ Componentes criadas:")
        print(f"      GHI máximo: {df['ghi'].max():.0f} W/m²")
        print(f"      DNI máximo: {df['dni'].max():.0f} W/m²")
        print(f"      DHI máximo: {df['dhi'].max():.0f} W/m²")
        
        return df
        
    except Exception as e:
        print(f"   ❌ Erro na decomposição: {e}")
        print(f"   🔄 Usando método alternativo simples...")
        
        # Método alternativo simples se DISC falhar
        # Utiliza correlações estatísticas básicas
        solar_position = solarposition.get_solarposition(df.index, lat, lon)
        
        # Estimativa conservadora baseada em análise de dados solariméricos
        # DNI ≈ 85% do GHI em condições médias (varia com clima local)
        clear_sky_dni_fraction = 0.85
        df['dni'] = df['ghi'] * clear_sky_dni_fraction
        df['dhi'] = df['ghi'] - df['dni'] * np.cos(np.radians(solar_position['zenith']))
        
        # Garante valores físicos válidos
        df['dni'] = df['dni'].clip(lower=0)
        df['dhi'] = df['dhi'].clip(lower=0, upper=df['ghi'])
        
        df['solar_zenith'] = solar_position['zenith']
        df['solar_azimuth'] = solar_position['azimuth']
        
        print(f"   ✅ Decomposição alternativa concluída:")
        print(f"      GHI máximo: {df['ghi'].max():.0f} W/m²")
        print(f"      DNI máximo: {df['dni'].max():.0f} W/m²")
        print(f"      DHI máximo: {df['dhi'].max():.0f} W/m²")
        
        return df

def test_orientations(df, lat, lon, orientations=None):
    """
    Testa diferentes orientações para encontrar a ótima
    
    Análise de sensibilidade para otimização de orientação:
    - Inclinação (tilt): ângulo com a horizontal [0-90°]
    - Azimute: orientação cardinal [0-360°, 180°=Norte no hemisfério sul]
    
    Trade-offs de projeto:
    - Horizontal (0°): máxima captação verão, menor no inverno
    - Latitude (≈24°): otimização anual teórica
    - Vertical (90°): captação uniforme, menor geração total
    
    Considerações para hemisfério sul:
    - 180° = Norte (máxima exposição solar)
    - 90° = Leste (geração matutina)
    - 270° = Oeste (geração vespertina)
    
    Parâmetros:
    -----------
    orientations : lista de tuplas (tilt, azimuth)
                  Se None, usa configurações típicas para análise
    """
    if orientations is None:
        # Configurações típicas para testar
        orientations = [
            (0, 180),           # Horizontal - máxima área de captação
            (abs(lat), 180),    # Latitude, Norte - ótimo teórico anual
            (abs(lat), 90),     # Latitude, Leste - geração matutina
            (abs(lat), 270),    # Latitude, Oeste - geração vespertina
            (90, 180),          # Vertical, Norte - máxima captação inverno
            (abs(lat)-10, 180), # Sub-ótimo: latitude - 10°
            (abs(lat)+10, 180), # Sobre-ótimo: latitude + 10°
        ]
    
    print(f"\n🔍 TESTE DE ORIENTAÇÕES:")
    print(f"   Testando {len(orientations)} configurações...")
    
    results = []
    
    for tilt, azimuth in orientations:
        # Simula cada orientação (sem prints detalhados para não poluir saída)
        try:
            # Chama simulação com orientação específica
            result = simulate_with_your_data(
                df, lat, lon, 
                module_power=550, num_modules=18,
                surface_tilt=tilt, surface_azimuth=azimuth
            )
            
            results.append({
                'tilt': tilt,
                'azimuth': azimuth, 
                'annual_generation': result['annual_generation'],
                'specific_yield': result['specific_yield'],
                'capacity_factor': result['capacity_factor']
            })
            
        except Exception as e:
            print(f"   ❌ Erro em {tilt}°/{azimuth}°: {e}")
            continue
    
    # Ordena por geração anual (critério de otimização principal)
    results.sort(key=lambda x: x['annual_generation'], reverse=True)
    
    print(f"\n📊 RANKING DE ORIENTAÇÕES:")
    print(f"   Rank | Inclin. | Azimute | Geração | Yield Esp. | Cap.Factor")
    print(f"   -----|---------|---------|---------|------------|----------")
    
    for i, r in enumerate(results[:7], 1):
        azimuth_name = {180: 'Norte', 90: 'Leste', 270: 'Oeste', 0: 'Sul'}.get(r['azimuth'], f"{r['azimuth']}°")
        print(f"   {i:2d}   |  {r['tilt']:4.0f}°  |  {azimuth_name:6s} | {r['annual_generation']:6.0f} | {r['specific_yield']:7.0f}  | {r['capacity_factor']*100:6.1f}%")
    
    print(f"\n🏆 ORIENTAÇÃO ÓTIMA:")
    best = results[0]
    print(f"   Inclinação: {best['tilt']}°")
    print(f"   Azimute: {best['azimuth']}° ({180: 'Norte', 90: 'Leste', 270: 'Oeste'}.get(best['azimuth'], 'Customizado'))")
    print(f"   Geração anual: {best['annual_generation']:,.0f} kWh")
    print(f"   Ganho vs horizontal: {((best['annual_generation']/results[-1]['annual_generation'])-1)*100:+.1f}%")
    
    return results

def analyze_your_data(df):
    """
    Análise específica dos seus dados
    
    Métricas de recurso solar e condições ambientais:
    - Irradiação anual: integral da irradiância para estimativa energética
    - Horas de insolação: período útil para geração fotovoltaica
    - Distribuição sazonal: variação mensal para análise de perfil
    - Condições térmicas: temperatura para cálculo de perdas
    - Velocidade do vento: coeficiente de transferência de calor
    
    Indicadores de qualidade:
    - Irradiação > 1600 kWh/m²/ano: recurso bom
    - Fração DNI/GHI > 0.7: favorece sistemas com seguimento
    - Amplitude térmica: impacto nas perdas por temperatura
    """
    print(f"\n📊 ANÁLISE DOS SEUS DADOS:")
    
    # Estatísticas gerais do dataset
    total_records = len(df)
    period_days = (df.index.max() - df.index.min()).days + 1
    
    print(f"   Total registros: {total_records:,}")
    print(f"   Período: {period_days} dias")
    print(f"   Frequência: {total_records/period_days:.0f} registros/dia")
    
    # GHI - Irradiância Global Horizontal (métrica fundamental)
    ghi_max = df['ghi'].max()                    # Irradiância de pico [W/m²]
    ghi_mean = df['ghi'].mean()                  # Irradiância média [W/m²]
    ghi_daylight = (df['ghi'] > 50).sum()        # Horas com sol (>50 W/m²)
    ghi_zero = (df['ghi'] == 0).sum()           # Horas noturnas
    
    print(f"\n☀️  GHI - IRRADIAÇÃO HORIZONTAL GLOBAL:")
    print(f"   Máxima: {ghi_max:.0f} W/m²")
    print(f"   Média geral: {ghi_mean:.0f} W/m²")
    print(f"   Horas com sol (>50 W/m²): {ghi_daylight:,}")
    print(f"   Horas noturnas (=0 W/m²): {ghi_zero:,}")
    
    # Média apenas durante horas de sol (mais representativa)
    if ghi_daylight > 0:
        ghi_mean_daylight = df[df['ghi'] > 50]['ghi'].mean()
        print(f"   Média durante horas de sol: {ghi_mean_daylight:.0f} W/m²")
    
    # Componentes DNI/DHI (decompostas pelo modelo DISC)
    print(f"\n🔬 COMPONENTES DECOMPOSTAS (PVLIB):")
    print(f"   DNI máximo: {df['dni'].max():.0f} W/m²")
    print(f"   DHI máximo: {df['dhi'].max():.0f} W/m²")
    # Proporção DNI/GHI indica potencial para tracking (>0.7 favorece)
    print(f"   Proporção DNI/GHI: {(df['dni'].mean() / df['ghi'].mean()):.2f}")
    
    # Elevação solar (validação astronômica dos dados)
    sun_max = df['sun_elevation'].max()          # Máxima elevação solar [graus]
    sun_positive = (df['sun_elevation'] > 0).sum()  # Horas com sol acima horizonte
    
    print(f"\n🌅 ELEVAÇÃO SOLAR:")
    print(f"   Máxima elevação: {sun_max:.1f}°")
    print(f"   Horas sol acima horizonte: {sun_positive:,}")
    
    # Temperatura ambiente (fundamental para perdas térmicas)
    temp_min = df['temp_air'].min()              # Temperatura mínima [°C]
    temp_max = df['temp_air'].max()              # Temperatura máxima [°C]
    temp_mean = df['temp_air'].mean()            # Temperatura média [°C]
    
    print(f"\n🌡️  TEMPERATURA:")
    print(f"   Mínima: {temp_min:.1f}°C")
    print(f"   Máxima: {temp_max:.1f}°C") 
    print(f"   Média: {temp_mean:.1f}°C")
    
    # Velocidade do vento (afeta coeficiente de transferência de calor)
    wind_mean = df['wind_speed'].mean()          # Velocidade média [m/s]
    wind_max = df['wind_speed'].max()            # Velocidade máxima [m/s]
    
    print(f"\n💨 VENTO:")
    print(f"   Velocidade média: {wind_mean:.1f} m/s")
    print(f"   Velocidade máxima: {wind_max:.1f} m/s")
    
    # Dados interpolados (qualidade dos dados PVGIS)
    interpolated_count = df['interpolated'].sum()
    interpolated_pct = (interpolated_count / len(df)) * 100
    
    print(f"\n🔍 QUALIDADE DOS DADOS:")
    print(f"   Registros interpolados: {interpolated_count:.0f} ({interpolated_pct:.1f}%)")
    print(f"   Registros originais: {len(df) - interpolated_count:.0f}")
    
    # Padrão mensal da irradiação (sazonalidade do recurso solar)
    monthly_ghi = df.groupby(df.index.month)['ghi'].mean()
    print(f"\n📅 GHI MÉDIO POR MÊS:")
    months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
             'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
    for month_num in monthly_ghi.index:
        month_name = months[month_num - 1]
        print(f"   {month_name}: {monthly_ghi[month_num]:.0f} W/m²")

def simulate_with_your_data(df, lat, lon, 
                           module_power=550, num_modules=18,
                           surface_tilt=None, surface_azimuth=180):
    """
    Simulação PV USANDO PVLIB COMPLETO
    
    ModelChain: cadeia integrada de modelos físicos
    1. Transposição de irradiância: Hay-Davies-Klucher-Reindl
       - Considera geometria solar e reflexão do solo
       - Modela corretamente irradiância circunsolar e difusa
    
    2. Temperatura da célula: modelo SAPM térmico
       - T_cell = T_air + (POA/1000) × ΔT
       - Considera convecção forçada pelo vento
    
    3. Modelo elétrico I-V: equação de diodo único
       - 5 parâmetros: I_L, I_0, Rs, Rsh, a
       - Correções por temperatura e irradiância
    
    4. Modelo do inversor: CEC (California Energy Commission)
       - Curva de eficiência η(P) dependente da carga
       - Perdas de conversão CC/CA realísticas
    
    Parâmetros:
    -----------
    df : DataFrame com dados meteorológicos processados
    lat, lon : coordenadas geográficas do sistema
    module_power : potência nominal STC do módulo [Wp]
    num_modules : número de módulos em série (configuração string)
    surface_tilt : inclinação do array [graus] (None = latitude)
    surface_azimuth : orientação do array [graus] (180 = Norte para hemisfério sul)
    """
    print(f"\n⚡ SIMULAÇÃO PV COM PVLIB MODELCHAIN:")
    
    # CONFIGURAÇÃO DO SISTEMA (agora parâmetros configuráveis)
    if surface_tilt is None:
        surface_tilt = abs(lat)  # Regra prática: inclinação ótima ≈ latitude local
    
    print(f"   Sistema: {num_modules} × {module_power}W = {num_modules * module_power / 1000:.1f} kWp")
    print(f"   🔧 ORIENTAÇÃO CONFIGURÁVEL:")
    print(f"      • Inclinação: {surface_tilt}° {'(latitude)' if surface_tilt == abs(lat) else '(customizada)'}")
    print(f"      • Azimute: {surface_azimuth}° (180° = Norte, 90° = Leste, 270° = Oeste)")
    print(f"   USANDO: Todos os modelos físicos da pvlib")
    
    # Localização pvlib com timezone correto
    pvlib_location = location.Location(lat, lon, tz='America/Sao_Paulo')
    
    # Módulo - do banco de dados pvlib ou customizado
    try:
        # Tentativa de usar banco CEC (California Energy Commission)
        # Base de dados com >20.000 módulos validados em laboratório
        cec_modules = pvsystem.retrieve_sam('CECMod')
        # Procura módulo similar ao especificado (Canadian Solar 54x células)
        suitable_modules = {k: v for k, v in cec_modules.items() 
                          if 'Canadian' in k and '54' in k}
        if suitable_modules:
            module_name = list(suitable_modules.keys())[0]
            module_parameters = suitable_modules[module_name]
            print(f"   📦 Módulo banco CEC: {module_name}")
        else:
            raise KeyError("Usando customizado")
    except:
        # Parâmetros customizados com parâmetros térmicos SAPM
        # Baseados em módulo Si-cristalino típico de 550Wp
        module_parameters = {
            # Coeficientes de temperatura (críticos para perdas térmicas)
            'alpha_sc': 0.0004,      # dI_sc/dT [A/°C] - corrente aumenta com temperatura
            'beta_oc': -0.0028,      # dV_oc/dT [V/°C] - tensão diminui com temperatura  
            'gamma_r': -0.0004,      # dP/dT [1/°C] - potência diminui ~0.04%/°C
            
            # Modelo de diodo único (5 parâmetros fundamentais)
            'a_ref': 1.8,            # Fator de idealidade modificado [V]
            'I_L_ref': 13.91,        # Fotocorrente STC [A] - proporcional à irradiância
            'I_o_ref': 3.712e-12,    # Corrente saturação reversa STC [A]
            'R_s': 0.348,            # Resistência série [Ω] - perdas ôhmicas
            'R_sh_ref': 381.68,      # Resistência paralelo STC [Ω] - correntes fuga
            
            # Especificações elétricas STC (1000 W/m², 25°C, AM1.5)
            'cells_in_series': 144,  # Células em série (topologia 12×12)
            'STC': module_power,     # Potência nominal [Wp]
            'V_oc_ref': 49.7,        # Tensão circuito aberto STC [V]
            'I_sc_ref': 13.91,       # Corrente curto-circuito STC [A]
            'V_mp_ref': 41.8,        # Tensão máxima potência STC [V]
            'I_mp_ref': 13.16,       # Corrente máxima potência STC [A]
            
            # Parâmetros térmicos SAPM necessários para modelo de temperatura
            'A0': -3.56, 'A1': -0.075, 'A2': 0.0, 'A3': 0.0, 'A4': 0.0,
            'B0': 0.0, 'B1': 0.0, 'B2': 0.0, 'B3': 0.0, 'B4': 0.0, 'B5': 0.0,
            'DTC': 3.0               # Delta T para SAPM [°C] - diferença T_cell - T_air
        }
        print(f"   📦 Módulo customizado: {module_power}W")
    
    # Inversor - do banco de dados pvlib com dimensionamento automático
    # Dimensionamento típico: P_ac_nominal = 0.8-0.9 × P_cc_array
    target_ac = num_modules * module_power * 0.85  # Define antes do try
    try:
        # Banco de dados CEC de inversores com eficiências validadas
        cec_inverters = pvsystem.retrieve_sam('cecinverter')
        # Dimensiona automaticamente baseado na potência CC
        suitable_inverters = {k: v for k, v in cec_inverters.items()
                            if 0.8 * target_ac <= v['Paco'] <= 1.2 * target_ac}
        if suitable_inverters:
            inverter_name = list(suitable_inverters.keys())[0]
            inverter_parameters = suitable_inverters[inverter_name]
            print("PA inversor")
            print(inverter_parameters)
            print(f"   🔌 Inversor banco CEC: {inverter_name}")
        else:
            raise KeyError("Usando customizado")
    except:
        # Parâmetros customizados para inversor string típico
        inverter_parameters = {
            'Paco': target_ac,                           # Potência CA nominal [W]
            'Pdco': num_modules * module_power * 0.95,   # Potência CC entrada [W]
            'Vdco': 360,                                # Tensão CC nominal [V]
            'Pso': 25,                                  # Potência autorconsumo [W]
            
            # Coeficientes curva eficiência CEC (polinômio de 4ª ordem)
            # η = (Pdc×(C0×Pdc + C1) + C2) / Pdc + C3×Pdc
            'C0': -0.000008, 'C1': -0.000120,
            'C2': 0.001400, 'C3': -0.020000,
            'Pnt': 0.02                                 # Potência noturna [W]
        }
        print(f"   🔌 Inversor customizado: {target_ac/1000:.1f}kW")
        
    losses_parameters = {
        'soiling': 2.0,          # Sujeira nos módulos (clima tropical) [%]
        'shading': 4.0,          # Sombreamento parcial (instalação otimizada) [%]  
        'mismatch': 2.5,         # Descasamento entre módulos [%]
        'wiring': 2.0,        # Perdas cabeamento CC (resistência, conectores) [%]
        #inversor,
        #outras perdas
    }
    
    # Sistema PV usando pvlib - configuração para SAPM
    system = pvsystem.PVSystem(
        surface_tilt=surface_tilt,                           # Inclinação [graus]
        surface_azimuth=surface_azimuth,                     # Orientação [graus]
        module_parameters=module_parameters,                 # Parâmetros elétricos módulo
        inverter_parameters=inverter_parameters,             # Parâmetros inversor
        modules_per_string=num_modules,                      # Módulos série por string
        strings_per_inverter=3,                              # Strings paralelo por inversor
        temperature_model_parameters=temperature_model_parameters,  # Modelo térmico SAPM
        losses_parameters=losses_parameters

    )
    
    print(system)

    
    # ModelChain - cadeia completa de modelos físicos integrados
    mc = modelchain.ModelChain(
        system=system,
        location=pvlib_location,
        losses_model='pvwatts'
        # Modelos default otimizados:
        # - Transposição: Hay-Davies-Klucher-Reindl  
        # - Temperatura: SAPM (definido no sistema)
        # - AOI: modelo físico para perdas por reflexão
        # - Spectral: correção por massa de ar
        # - DC model: diodo único com 5 parâmetros
        # - AC model: CEC com curva de eficiência
        # - Losses: sem perdas adicionais por simplicidade
    )
    
    print(mc)
    
    
    print(f"   🔧 ModelChain pvlib configurado:")
    print(f"      • Temperatura: SAPM (modelo térmico físico)")
    print(f"      • Perdas: modelo padrão (reflexão, espectral, etc)")
    
    # EXECUTA SIMULAÇÃO PVLIB
    print(f"   🔄 Executando ModelChain...")
    mc.run_model(df)
    
    # Extrai resultados da simulação completa
    ac_power_total = mc.results.ac    # Potência CA total do sistema [W]
    dc_power_total = mc.results.dc    # Potência CC total do sistema [W]
    
    # Comentário: linhas para debug se necessário
#     print(mc)                        # Detalhes do ModelChain
#     print(mc.results)               # Resultados disponíveis
#     print(ac_power_total.head())    # Primeiros valores de potência CA
#     print(dc_power_total.head())    # Primeiros valores de potência CC
    
    # Cálculo das métricas de performance (usando potência customizada)
    annual_generation_kwh = ac_power_total.sum() / 1000         # Geração anual [kWh]
    peak_power_kw = ac_power_total.max() / 1000                 # Potência pico atingida [kW]
    # Fator de capacidade: geração real vs teórica máxima
    capacity_factor = annual_generation_kwh / (num_modules * module_power * 8760 / 1000)
    # Yield específico: geração por kWp instalado
    specific_yield = annual_generation_kwh / (num_modules * module_power / 1000)
    # Eficiência média do inversor
    inverter_efficiency = (ac_power_total.sum() / dc_power_total.sum())
    
    print(f"\n📊 RESULTADOS DA SIMULAÇÃO PVLIB:")
    print(f"   Geração anual: {annual_generation_kwh:,.0f} kWh")
    print(f"   Potência pico: {peak_power_kw:.2f} kW")
    print(f"   Capacity Factor: {capacity_factor * 100:.1f}%")
    print(f"   Yield específico: {specific_yield:.0f} kWh/kWp/ano")
    print(f"   Eficiência média inversor: {inverter_efficiency}")
    
    # Performance Ratio (PR) - métrica padrão IEC 61724
    # PR = Energia_real / Energia_teórica_STC (considera todas perdas exceto irradiância)
    pr = None  # Inicializa variável
    if hasattr(mc.results, 'total_irrad'):
        # Irradiância no plano do array (POA - Plane of Array)
        poa_global = mc.results.total_irrad['poa_global']
        # Energia teórica: POA × potência_nominal × eficiência_STC
        theoretical = (poa_global * num_modules * module_power / 1000).sum() / 1000
        pr = annual_generation_kwh / theoretical if theoretical > 0 else 0
        print(f"   Performance Ratio: {pr:.3f}")
        
        # Ganho de transposição: POA vs GHI (benefício da inclinação)
        poa_mean = poa_global.mean()
        ghi_mean = df['ghi'].mean()
        transposition_gain = (poa_mean / ghi_mean - 1) * 100
        print(f"   Ganho transposição: {transposition_gain:+.1f}%")
    
    # Análise mensal usando resultados pvlib (sazonalidade da geração)
    monthly_gen = ac_power_total.groupby(ac_power_total.index.month).sum() / 1000
    print(f"\n📅 GERAÇÃO MENSAL (PVLIB):")
    months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
             'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
    
    for month_num in monthly_gen.index:
        month_name = months[month_num - 1]
        print(f"   {month_name}: {monthly_gen[month_num]:.0f} kWh")
    
    # Estatísticas operacionais do sistema
    operating_hours = (ac_power_total > 10).sum()  # >10W considerado operando
    avg_daily_gen = annual_generation_kwh / 365     # Geração diária média
    
    print(f"\n📈 ESTATÍSTICAS OPERACIONAIS:")
    print(f"   Horas de operação: {operating_hours:,} ({operating_hours/len(df)*100:.1f}%)")
    print(f"   Geração média diária: {avg_daily_gen:.1f} kWh")
    
    # Temperatura célula (análise térmica com pvlib)
    if hasattr(mc.results, 'cell_temperature'):
        cell_temp_mean = mc.results.cell_temperature.mean()  # Temperatura média [°C]
        cell_temp_max = mc.results.cell_temperature.max()    # Temperatura máxima [°C]
        print(f"   Temp célula média (pvlib): {cell_temp_mean:.1f}°C")
        print(f"   Temp célula máxima (pvlib): {cell_temp_max:.1f}°C")
    
    return {
        'annual_generation': annual_generation_kwh,
        'monthly_generation': monthly_gen.to_dict(),
        'ac_power': ac_power_total,
        'dc_power': dc_power_total,
        'capacity_factor': capacity_factor,
        'specific_yield': specific_yield,
        'performance_ratio': pr,
        'pvlib_results': mc.results
    }

# Executar processamento completo
if __name__ == "__main__":
    print("🚀 INICIANDO PROCESSAMENTO DOS SEUS DADOS PVGIS COM PVLIB...")
    print("=" * 60)
    print("🔧 CONFIGURAÇÕES DISPONÍVEIS:")
    print("   • module_power: Potência do módulo em W")
    print("   • num_modules: Quantidade de módulos")
    print("   • surface_tilt: Inclinação em graus (None = latitude ótima)")
    print("   • surface_azimuth: Orientação (180° = Norte, 90° = Leste, 270° = Oeste)")
    print("=" * 60)
    
    results = process_your_pvgis_structure()
    
    if results is not None:
        print(f"\n✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!")
        print(f"   Dados PVGIS processados com ModelChain da pvlib")
        print(f"   Todos os modelos físicos aplicados")
        print(f"   Resultados validados e precisos!")
        
        print(f"\n" + "="*60)
        print("💡 DICAS DE USO:")
        print("   1. Modifique surface_tilt e surface_azimuth na função")
        print("   2. Descomente test_orientations() para otimização")
        print("   3. Ajuste module_power e num_modules conforme necessário")
        print("="*60)
        
    else:
        print(f"\n❌ Erro no processamento")

INFO:__main__:Buscando dados PVGIS para (-23.761759, -53.329072) - ano 2020


🚀 PROCESSADOR PVGIS REFATORADO COM CORREÇÕES
🔧 PROCESSADOR PVGIS REFATORADO
📍 Coordenadas: -23.761759, -53.329072


INFO:__main__:Processados 8,784 registros - Período: 2019-12-31 21:03:00-03:00 até 2020-12-31 20:03:00-03:00
INFO:__main__:Decomposição concluída - GHI máx: 1134, DNI máx: 982, DHI máx: 539 W/m²


✅ Recebidos 8,784 registros

📊 PRIMEIROS 3 REGISTROS:
   Registro 1: {'time': '20200101:0003', 'G(i)': 0.0, 'H_sun': 0.0, 'T2m': 25.02, 'WS10m': 1.93, 'Int': 0.0}
   Registro 2: {'time': '20200101:0103', 'G(i)': 0.0, 'H_sun': 0.0, 'T2m': 24.85, 'WS10m': 1.59, 'Int': 0.0}
   Registro 3: {'time': '20200101:0203', 'G(i)': 0.0, 'H_sun': 0.0, 'T2m': 24.73, 'WS10m': 1.17, 'Int': 0.0}

🔧 PROCESSANDO DADOS PARA PVLIB:

🔬 DECOMPOSIÇÃO GHI → DNI/DHI:

📊 ANÁLISE DOS DADOS:
   Total registros: 8,784
   GHI máximo: 1134 W/m²
   GHI médio: 223 W/m²
   Temp. média: 23.3°C
   Vento médio: 2.3 m/s

🎯 SIMULAÇÃO PRINCIPAL:

⚡ SIMULAÇÃO PV COM PVLIB:
   Sistema: 18 × 550W = 9.9 kWp
   Inclinação: 24°, Azimute: 180°

📊 RESULTADOS:
   Geração anual: 3,343 kWh
   Capacity Factor: 3.9%
   Yield específico: 338 kWh/kWp/ano

🔍 OTIMIZAÇÃO DE ORIENTAÇÃO:

🔍 TESTE DE ORIENTAÇÕES:
   Testando 7 configurações...

📊 RANKING DE ORIENTAÇÕES:
   Rank | Inclin. | Azimute | Geração | Yield Esp. | Cap.Factor
   -----|-----